In [ ]:
import pandas as pd
import re
import requests
import urllib
import nltk
from nltk.corpus import stopwords
import pymorphy2
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

#nltk.download('stopwords')
#nltk.download('punkt')

In [ ]:
separator = r'\', \''
frame_columns_names = ["Тема обращения","Суть обращения",
                        "Чиновник","Должность",
                        "Дата подачи заявления"]
df = pd.read_csv('Mosru_dump_all_pages.csv',
                    encoding = 'utf-8',
                    sep = separator,
                    names = None,
                    skiprows = 1)
df.columns = frame_columns_names

In [ ]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def clean_string(string):
    dirt_patterns = ['\'','\\n','\n','\\\\/S+','\\\\','/\S+','\(.*\)']
    for pattern in dirt_patterns:
        string = re.sub(pattern,'',string)
    return string

In [ ]:
def search_theme(string):
    res = re.findall('«.*»',string)
    return res[0] if len(res)>0 else ""

In [ ]:
def search_address(string):
    res = re.findall('по адресу\s?(.*$)',string)
    return res[0] if len(res)>0 else ""

In [ ]:
def preprocessingText(line):
    line = line.lower()
    line = re.sub(r'\d+','',line,flags=re.UNICODE)#delete numbers
    line = re.sub(r'[^\w\s]',' ',line,flags=re.UNICODE)
    tl = nltk.word_tokenize(line) #tl - tokenize line
    nft = [morph.parse(token)[0].normal_form for token in tl if len(token)>1]#nft - normal form token
    clean_words_list = [normalToken for normalToken in nft if normalToken not in stop_words and len(normalToken)>1]#преобразованный список
    return ' '.join(clean_words_list)

In [ ]:
for column in frame_columns_names:
    df[column] = df[column].apply(clean_string)

In [ ]:
df["theme"] = df['Тема обращения'].apply(search_theme)

In [ ]:
df["address"] = df['Тема обращения'].apply(search_address)

In [ ]:
df['Obr_clean'] = df['Суть обращения'].apply(preprocessingText)

In [ ]:
df.head(2)

In [ ]:
df.to_csv('clean_frame.csv',sep='|',encoding='utf-8')

Вторая часть после загрузки и обработки данных (подтянуть адреса c помощью mapbox api)

In [55]:
n_df = pd.read_csv('clean_frame.csv', sep='|', encoding='utf-8')

In [ ]:
unique_addresses = n_df['address'].unique().tolist()

In [ ]:
mapbox_token = "pk.eyJ1IjoidmFkaW05NiIsImEiOiJja25nMGowN2QyNnFlMnFtdTkzYjBjejdkIn0.18dpTc8Vgja2gPDUYgrSkw"

In [ ]:
def search_district(address):
    url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{0}.json?access_token={1}".format(urllib.parse.quote("Москва "+str(address)), mapbox_token)
    response = requests.get(url)
    district = ""
    try:
        district = response.json()['features'][0]['place_name']
    except Exception as error:
        print("Возникла ошибка-{0}, адрес - {1}".format(str(error),str(address)) )
    finally:
        return district

In [ ]:
with open('unique_addresses_frame.csv','w',encoding='utf-8') as file:
    file.write('Адрес|Полный ответ\n')
    for address in tqdm(unique_addresses):
        full_response = search_district(address)
        file.write('{0}|{1}\n'.format(address,full_response))

Схлестываем значения адресов с "чистым фреймом"

In [ ]:
addresses_frame = pd.read_csv('unique_addresses_frame.csv',encoding='utf-8',sep='|')

Тематическое моделирование

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('clean_frame.csv',encoding='utf-8',sep='|')

In [ ]:
df.head()

In [ ]:
df['Obr_clean'][:10]

In [ ]:
data = df['Obr_clean'].values.tolist()

In [ ]:
data_words2 = [nltk.word_tokenize(str(line)) for line in tqdm(data)]

In [ ]:
id2word = corpora.Dictionary(data_words2)
texts = data_words2
#term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
#количество тем
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word=id2word, num_topics = num_topics)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
LDAvis_prepared = pyLDA.gensim